In [1]:
import pyrealsense2 as rs2
import open3d as o3d
import numpy as np
import time
from spygrt.stream import Recording, NoMoreFrames, DualRecording
from spygrt.tracker import Tracker


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
rec1 = Recording('E:/VISION/129/Recording_Session/121622061907_2023Feb13022044.bag')
rec2 = Recording('E:/VISION/129/Recording_Session/123122061443_2023Feb13022044.bag')
rec1.load_calibration('121622061907_cal.txt')
rec2.load_calibration('123122061443_cal.txt')



array([[-0.76596981, -0.47690493,  0.46312463, -0.43199882],
       [-0.65983385,  0.45945311, -0.61818653,  0.47658443],
       [ 0.0808824 , -0.76817775, -0.65726215,  0.49703872],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [3]:
d = DualRecording(rec1,rec2)
d.start_stream()
d.get_frames()

((<pyrealsense2.frame Z16 #1160 @1676258444174.644043>,
  <pyrealsense2.frame RGB8 #1194 @1676258444174.980957>),
 (<pyrealsense2.frame Z16 #1121 @1676258444195.972412>,
  <pyrealsense2.frame RGB8 #1123 @1676258444196.309326>))

In [4]:
t = Tracker(ref_surface=d.pcd)

[Open3D INFO] Picked point #232123 (0.095, -0.062, 0.14) to add in queue.
[Open3D INFO] Picked point #1160248 (-0.035, 0.052, 0.16) to add in queue.


In [10]:
d.compute_pcd(True)

PointCloud on CPU:0 [1843200 points (Float32)].
Attributes: colors (dtype = UInt8, shape = {1843200, 3}).

In [18]:
a = time.time()
t.get_motion(d.compute_pcd(True))
b= time.time()

In [19]:
a-b

-0.6968114376068115

In [5]:
pcd = o3d.t.geometry.PointCloud(o3d.core.Device('CUDA:1'))

In [47]:
pcd.cuda()

PointCloud on CUDA:0 [0 points].
Attributes: None.

In [25]:
pcd.cpu()

PointCloud on CPU:0 [0 points].
Attributes: None.

In [26]:
pcd

PointCloud on CUDA:0 [0 points].
Attributes: None.

In [9]:
rec1._rs_pc.map_to(rec1.frame[1])

In [10]:
dat=rec1._rs_pc.calculate(rec1.frame[0])

In [11]:
depth_dat = np.asanyarray(dat.get_vertices()).view(np.float32).reshape([-1, 3])

In [12]:
col_dat = np.asanyarray(rec1.frame[1].get_data()).reshape([-1, 3])

In [13]:
pcd.point["positions"] = depth_dat

In [14]:
pcd.point["colors"] = col_dat

In [15]:
pcd.device

CUDA:1

In [16]:
pcd

PointCloud on CUDA:1 [921600 points (Float32)].
Attributes: colors (dtype = UInt8, shape = {921600, 3}).

In [17]:
pcd.cpu().cuda()

RuntimeError: [Open3D Error] (class std::shared_ptr<class open3d::core::MemoryManagerDevice> __cdecl open3d::core::MemoryManager::GetMemoryManagerDevice(const class open3d::core::Device &)) D:\a\Open3D\Open3D\cpp\open3d\core\MemoryManager.cpp:142: Unsupported device "CUDA:0". Set BUILD_CUDA_MODULE=ON to compile for CUDA support and BUILD_SYCL_MODULE=ON to compile for SYCL support.


AttributeError: module 'open3d' has no attribute 'cuda'